# Project outline

Objective: predict % of silica in the iron ore concentrate.

Attention points:
* Data is not sampled in uniform frequency
* Columns 2 and 3 are before feeding into the process (consider retention time)
* Columns 4:8 are relevant quality measurements in the end of the process (probably doesnt need retention time)
* Columns 9:22 are process data
* Last column is % of silica in the iron ore concentrate and is a lab measurement


### Packages imports

In [2]:
import pandas as pd

### Data import

In [4]:
df = pd.read_csv('../data/MiningProcess_Flotation_Plant_Database.csv', decimal=',')  # Adjusts data format
df.columns = df.columns.str.lower().str.replace(' ', '_')  # improves data manipulation
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

df.head(2)

,%_iron_feed,%_silica_feed,starch_flow,amina_flow,ore_pulp_flow,ore_pulp_ph,ore_pulp_density,flotation_column_01_air_flow,flotation_column_02_air_flow,flotation_column_03_air_flow,...,flotation_column_07_air_flow,flotation_column_01_level,flotation_column_02_level,flotation_column_03_level,flotation_column_04_level,flotation_column_05_level,flotation_column_06_level,flotation_column_07_level,%_iron_concentrate,%_silica_concentrate
date,,,,,,,,,,,,,,,,,,,,,
2017-03-10 01:00:00,55.2,16.98,3019.53,557.434,395.713,10.0664,1.74,249.214,253.235,250.576,...,250.884,457.396,432.962,424.954,443.558,502.255,446.370,523.344,66.91,1.31
2017-03-10 01:00:00,55.2,16.98,3024.41,563.965,397.383,10.0672,1.74,249.719,250.532,250.862,...,248.994,451.891,429.560,432.939,448.086,496.363,445.922,498.075,66.91,1.31


In [5]:
df.isna().sum()

%_iron_feed                     0
%_silica_feed                   0
starch_flow                     0
amina_flow                      0
ore_pulp_flow                   0
ore_pulp_ph                     0
ore_pulp_density                0
flotation_column_01_air_flow    0
flotation_column_02_air_flow    0
flotation_column_03_air_flow    0
flotation_column_04_air_flow    0
flotation_column_05_air_flow    0
flotation_column_06_air_flow    0
flotation_column_07_air_flow    0
flotation_column_01_level       0
flotation_column_02_level       0
flotation_column_03_level       0
flotation_column_04_level       0
flotation_column_05_level       0
flotation_column_06_level       0
flotation_column_07_level       0
%_iron_concentrate              0
%_silica_concentrate            0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 737453 entries, 2017-03-10 01:00:00 to 2017-09-09 23:00:00
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   %_iron_feed                   737453 non-null  float64
 1   %_silica_feed                 737453 non-null  float64
 2   starch_flow                   737453 non-null  float64
 3   amina_flow                    737453 non-null  float64
 4   ore_pulp_flow                 737453 non-null  float64
 5   ore_pulp_ph                   737453 non-null  float64
 6   ore_pulp_density              737453 non-null  float64
 7   flotation_column_01_air_flow  737453 non-null  float64
 8   flotation_column_02_air_flow  737453 non-null  float64
 9   flotation_column_03_air_flow  737453 non-null  float64
 10  flotation_column_04_air_flow  737453 non-null  float64
 11  flotation_column_05_air_flow  737453 non-null  float64
 12  flotation_